<a href="https://colab.research.google.com/github/tpyte001/comput_phy/blob/main/PHY411ReportTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>Adaptive Methods for Trapezoidal Rule<h1>
<h2>By Taylor Pytel<h2>

</center>



---



The trapezoid rule is a quick and easy way to approximate the integrals of hard to calculate functions, but if we have a function whose domain includes areas of rapid change, then the trapezoid rule will start to suffer for accuracy. When this is the case, it's better to use the method described below.

First, lets define a function whose integral we can calculate by hand to serve as an example:

Let: 


> $f(x)=x \sin^2(x)$








Then the Integral of $f(x)$ is given by:
> $I = \int_{a}^{b} x \sin^2(x) dx $


> $I = \large{ \frac{x^2}{4}} - \frac{1}{4} x \sin(2x) -\frac{1}{8} \cos(2x)$





Evaluated from: 
>$a=1$ and $b=2$

>$I|_{1,2} = \frac{1}{8} [6+2 \sin(2) - 4 \sin(4) + \cos(2) - \cos(4)]$

>$I|_{1,2} = 1.3854$



Adaptive Trapezoidal Method:

In [2]:
import numpy as np
f = lambda x: x*(np.sin(x))**2
def trap_adapt(f,a,b,epsilon=1.0e-8):
    def step(x1,x2,fx1,fx2):
        x_mid = (x1 + x2)/2.0
        fx_mid = f(x_mid)
        h1 = x2 - x1
        h2 = h1/2.0
        I1 = (fx1 + fx2) * h1/2.0
        I2 = (fx1 + 2*fx_mid + fx2) * h2/2.0
        error = abs((I2-I1)/3.0)
        if error <= h2*delta:
            return I2
        else:
            return step(x1,x_mid,fx1,fx_mid)+step(x_mid,x2,fx_mid,fx2)
    delta = epsilon/(b-a)
    fa = f(a)
    fb = f(b)
    return step(a,b,fa,fb)

print(trap_adapt(f,1,2))

1.3854127004760155


This program works as follows:

Let $f:R \to R \ $ be defined by $f(x)$, where $f(x)$ is a function whose integral must be approximated. 

The above code can make such an approximation via defining the step size recursively. The stepsize is given its own error tolerance, which is checked after every iteration, allowing for adjustments over intervals which require more percise measurements. 

On the first iteration, the function `trap_adapt` takes in the parameters $\{f(x), a, b, ϵ\}$, using them to calculate the following:

* $δ = { ϵ \over (b-a)}$ ; the target accuracy per step per domain of integration.  
* $f(a)$ The image of a under $f$.
* $f(b)$ The image of b under $f$.

The values calcualted above $\{ a, b, f(a), f(b) \}$  are then passed as parameters $\{a=x_1,b=x_2,f(a)=f(x_1),f(b)=f(x_2)\}$ into `step`; The function embedded within `trap_adapt`.

Inside `step` $\{x_1,x_2,f(x_1),f(x_2)\}$ are used to get the following values:

* The midpoint of the interval of integration, with lower and upper bounds $x_1$ and $x_2$ such that $x_{mid}={x_1 + x_2 \over 2}$.
* The  image of the midpoint under $f$ at $f(x_{mid})=$  $f(\frac {x_1 + x_2}{2})$.
* An approximation via trapezoid rule with a single step $I_1 = \frac{1}{2} [f(x_1) + f(x_2)] b - a$.
* The domain of integration is split along $f(x_{mid})$ into two seperate integrals, each of width $b-a \over 2$, both intervals are then approximated with the trapezoid rule: $I_2 = \frac{1}{2} [f(x_1) + 2f(x_{mid}) + f(x_2)] \frac{b-a}{2}$ where $I_2$ is the sum of both integrals.

We now have values $\{ x_{mid},f(x_{mid}),I_1,I_2 \}$
which we can use to calculate the error: $\frac{1}{3}| I_2 - I_1 |$ which is then checked against the target error of each respective integral: $hδ$. This is the part that makes this method powerful. The error of each respective integral is checked against our error tolerance in order to see if the step size needs adjusting:

* If $hδ \ge  |\frac{1}{3}I_2 - I_1 |$ is true, then `step` function returns $I_2$, as the integral of $f(x)$ is succesfully approximated for that interval.

* If $hδ \ge  |\frac{1}{3}I_2 - I_1 |$ is false, then `step` uses recursion, reiterating the above process for smaller subdivisions of intervals over the interval which returned false. 

If the interval returns false, the profram essentially splits that specific interval of integration up into a sum of smaller integrals, repeating the above process for each area of the domain of integration that requires it untill our error tolerance is met. 

Some areas of $f$ will be be true while others false, depending on how quickly the change between respective images. A steep change (meaning a large amount of verticle distance between $f(x)$ and $f(x + Δx)$, over a small change in horizontal distance $x$ and $x + Δx$) will require smaller and more accurate domains of integration, spanning the interval in question.

Your given error tolerance is used to determine wether or not a change in verticle distance is steep enough to merit modifying the step size for better results. 

The idea of this program is to essentially takes the integral of $f(x)$, split it into two integrals, sum those, then if nessecary split that interval into two more integrals and sum those, and so on and so forth, relative to a specific interval of integraiton. Some parts of f(x) will pass while others will fail, but you are never working with more than 2 steps at a time. Instead of increasing the number of steps, you are essentially breaking down the function into several integrals and summing up all their domains of integration, then returing that sum as your approximation.